In [4]:
import sys
sys.path.append('../utils/')
from utils import *

eu = ETL_Utils()
# eu.meta_sheet_info(excel_name = '')
# eu.target_sheet_list = ['Warehouse_inventory_On_Hand_WMS']
# eu.read_excel('../data/dummy/onhand_sap.xlsx')

In [ ]:
eu = ETL_Utils()
eu.connect_azuredb()
sql = '''
    SELECT 
        t1.* 
    FROM (
        SELECT  
            *, 
            ROW_NUMBER () over (partition by Week, FactoryCd, ProductGroup1, ProductGroup2, ProductGroup3, ProductName, SalesEntity , Segment, Month, Updated_Date Order by Updated_Date) AS RN 
        from ALLOCATION_PLAN
        where Updated_Date  =  (select max(Updated_Date) from ALLOCATION_PLAN)
    ) as t1
    WHERE 
    t1.RN = 1
'''
eu.alloc_df = eu.fetch_data(sql)
eu.conn.close()

In [13]:
import pandas as pd

df = pd.read_csv('./Calendar.csv', encoding = 'cp949').dropna(how = 'all', axis = 1)
df = df.dropna(how = 'all', axis = 0)

In [14]:
eu = ETL_Utils()
eu.insert_dataframe('BI_CALENDAR', df)

[EVENT] INSERT INFO: 
 	 TABLE NAME:  BI_CALENDAR 
 	 TABLE COLUMNS ARE BELOW ['날짜Key', '년도번호', '년도명', '월번호', '월명', '일번호', '일명', '분기번호', '분기명', '주차번호', '주차명', '요일번호', '요일명', '년중일수', 'S년도월', 'S년도분기', 'S년도분기월', 'S년도주차', '날짜AK1', '월AK1']


In [12]:
eu.conn.close()

In [15]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)